In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
from gensim.models import Word2Vec
skipgram = Word2Vec.load('w2vtrain300.bin')

In [14]:
print("The total number of words are : ",len(skipgram.wv))

The total number of words are :  62269


In [15]:
skipgram.wv.similarity('chính_sách','pháp_luật')

0.31097564

In [16]:
data = pd.read_csv('final_data.csv')

In [17]:
stringdata=[]
for i in range(len(data)):
    stringdata.append(str(data.Title_processed[i])+' '+str(data.Content_processed[i]))

In [18]:
data['Title_Content_processed']=stringdata

In [19]:
data.head()

,Unnamed: 0.1,Unnamed: 0,Title_processed,Content_processed,label,Length,Title_Content_processed
0,0,1,thúc_đẩy quan_hệ hợp_tác quốc_hội việt_nam nhâ...,tổng_bí_thư nguyễn_phú_trọng hội_kiến chủ_tịch...,1,274,thúc_đẩy quan_hệ hợp_tác quốc_hội việt_nam nhâ...
1,1,2,tổng_bí_thư nguyễn_phú_trọng hội_kiến chủ_tịch...,tổng_bí_thư nguyễn_phú_trọng hội_kiến chủ_tịch...,1,229,tổng_bí_thư nguyễn_phú_trọng hội_kiến chủ_tịch...
2,2,3,giám_đốc công_an hà_nội vụ_án rửa tiền ước_tín...,thảo_luận hội_trường quốc_hội dự_thảo luật_phò...,1,229,giám_đốc công_an hà_nội vụ_án rửa tiền ước_tín...
3,3,4,ông võ_văn_thưởng hội_đàm trực_tuyến bí_thư ba...,thường_trực_ban_bí_thư võ_văn_thưởng hội_đàm ủ...,1,79,ông võ_văn_thưởng hội_đàm trực_tuyến bí_thư ba...
4,4,5,tổng_bí_thư nguyễn_phú_trọng hội_kiến thủ_tướn...,tổng_bí_thư nguyễn_phú_trọng hội_kiến thủ_tướn...,1,299,tổng_bí_thư nguyễn_phú_trọng hội_kiến thủ_tướn...


In [20]:
maxi=-1
for i,rev in enumerate(data['Title_Content_processed']):
    tokens=rev.split()
    if(len(tokens)>maxi):
        maxi=len(tokens)
print(maxi)

1085


In [21]:
def FunctionText2Vec_N(inpTextData):
    W2Vec_Data=[]
    for i in inpTextData:
        Sentence = []
        WordsVocab=str(i).split()
        for j in range(1050):
            try:
                Sentence.append(skipgram.wv[WordsVocab[j]])
            except:
                Sentence.append(np.zeros(300))       
        W2Vec_Data.append(Sentence)     
    return(W2Vec_Data)

In [46]:
x_train,x_test,y_train,y_test=train_test_split(data.Title_Content_processed,data.label,test_size=0.30,random_state=42)

In [47]:
x_train_np= data_Vector=np.array(FunctionText2Vec_N(x_train))

In [48]:
x_test_np= data_Vector=np.array(FunctionText2Vec_N(x_test))

In [49]:
x_train_np.shape

(1405, 1050, 300)

In [50]:
x_test_np.shape

(603, 1050, 300)

In [27]:
from torch import nn
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error as mape

In [28]:
from keras.layers import Dense, Dropout, LSTM, Embedding, Flatten
from keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences



In [29]:
import keras

In [57]:
model=Sequential()

In [59]:
model.add(LSTM(128,input_shape=(x_train_np.shape[1],x_train_np.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(1 ,activation='sigmoid', return_sequences=True))

ValueError: Input 0 of layer "lstm_10" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 128)

In [55]:
# embedding_vector_features=45

# model=Sequential()

# model.add(Embedding(15275,embedding_vector_features,input_length=1050))

# model.add(LSTM(128,input_shape=(x_train_np.shape[1],x_train_np.shape[2]),activation='relu',return_sequences=True))

# model.add(Dropout(0.2))

# model.add(LSTM(128,activation='relu'))

# model.add(Dropout(0.2))

# # for units in [128,128,64,32]:

# # model.add(Dense(units,activation='relu'))

# # model.add(Dropout(0.2))

# model.add(Dense(32,activation='relu'))

# model.add(Dropout(0.2))

# model.add(Dense(4,activation='softmax'))

# model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [53]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1050, 45)          687375    
                                                                 
 lstm_2 (LSTM)               (None, 1050, 128)         89088     
                                                                 
 dropout_2 (Dropout)         (None, 1050, 128)         0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 32)                4128      
                                                                 
 dropout_4 (Dropout)         (None, 32)               

In [56]:
model.fit(x_train_np,y_train,validation_data=(x_test_np,y_test),epochs=120,batch_size=64)

Epoch 1/120


ValueError: in user code:

    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 993, in train_s

In [38]:
# model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.RMSprop(lr=1e-2),metrics=['accuracy'])

In [39]:
print("Fitting model >>")
hist = model.fit(x_train_np,y_train,batch_size=64,epochs=30,validation_split=0.1,verbose=1)

Fitting model >>
Epoch 1/30


ValueError: in user code:

    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 994, in train_s